In [54]:
import numpy as np
import cv2 as cv


def pre_pro(pic_name, x0, y0):
    # 打开图片
    img = cv.imread(pic_name)
    # 灰度化
    gray_img = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
    # 二值化
    # 此时的第二个和第三个参数，
    # 即二值化的上下阈值需要根据图片的实际情况个来调整
    ret, binary_img = cv.threshold(gray_img, 180, 255, cv.THRESH_BINARY)
    print("binary_img")
    print(binary_img)
    print(binary_img.shape)
    cv.imshow('binary_img', binary_img)
    cv.waitKey(0)
    cv.destroyAllWindows()

    # 图像像素按行和列求和
    # 二值图像反相
    con_bi_img = 255 - binary_img
    # print(np.shape(binary_img))
    cv.imshow('con_bi_img', con_bi_img)
    cv.waitKey(0)
    cv.destroyAllWindows()
    column_sum_img = np.sum(con_bi_img, axis=0)
    row_sum_img = np.sum(con_bi_img, axis=1)
    # 排序
    sort_column_sum = np.sort(column_sum_img)
    sort_column_sum_indices = np.argsort(column_sum_img)
    sort_row_sum = np.sort(row_sum_img)
    sort_row_sum_indices = np.argsort(row_sum_img)
    print("列：\n")
    print(sort_column_sum[len(sort_column_sum) - 5:])
    print(sort_column_sum_indices[len(sort_column_sum_indices) - 5:])
    print("行：\n")
    print(sort_row_sum[len(sort_row_sum) - 5:])
    print(sort_row_sum_indices[len(sort_row_sum_indices) - 5:])

    # 通过每一列和每一行像素和的输出值，判断坐标轴的位置
    # 在坐标轴范围内遍历每一列，将曲线下面的部分设置成黑色
    for i in range(x0, binary_img.shape[1]):
        flag = 0
        for j in range(0, binary_img.shape[0]):
            if binary_img[j][i] == 0:
                flag += j
                break
        for j in range(flag, y0):
            binary_img[j][i] = 0
    cv.imshow('binary_img', binary_img)
    cv.waitKey(0)
    cv.destroyAllWindows()
    return binary_img

In [55]:
import numpy as np
from numpy.core.fromnumeric import shape
import cv2 as cv

def get_points(picture_name, xMax, xMin, yMax, yMin, x0, y0):

    binary_img = pre_pro(picture_name, x0, y0)
    cv.imshow('binary_img', binary_img)
    cv.waitKey(0)
    cv.destroyAllWindows()

    # 边缘提取
    xgrd = cv.Sobel(binary_img, cv.CV_16SC1, 1, 0)
    ygrd = cv.Sobel(binary_img, cv.CV_16SC1, 0, 1)

    egde_output = cv.Canny(xgrd, ygrd, x0, y0)
    cv.imshow('canny_edge', egde_output)
    cv.waitKey(0)
    cv.destroyAllWindows()

    # 图像像素按行和列求和
    column_sum_img = np.sum(egde_output, axis=0)
    row_sum_img = np.sum(egde_output, axis=1)
    # 排序
    sort_column_sum = np.sort(column_sum_img)
    sort_column_sum_indices = np.argsort(column_sum_img)
    sort_row_sum = np.sort(row_sum_img)
    sort_row_sum_indices = np.argsort(row_sum_img)
    print(sort_column_sum[len(sort_column_sum) - 10:])
    print(sort_column_sum_indices[len(sort_column_sum_indices) - 10:])
    print(sort_row_sum[len(sort_row_sum) - 10:])
    print(sort_row_sum_indices[len(sort_row_sum_indices) - 10:])

    fc = egde_output[0:binary_img.shape[0], 0:binary_img.shape[1]]
    cv.imshow('function_curve', fc)
    cv.waitKey(0)
    cv.destroyAllWindows()

    # 提取图像数据
    rows = (fc.shape)[0]
    cols = (fc.shape)[1]

    min_x = xMin
    max_x = xMax
    min_y = yMin
    max_y = yMax

    x_axis = np.empty([rows, cols])
    y_axis = np.empty([cols, rows])

    # x_interval和y_interval用于调整数据长度，在报错的时候可以通过他们来调整
    x_interval = (max_x - min_x) / (cols + 1)
    x_value = np.arange(min_x + x_interval, max_x, x_interval)
    # 自动调整长度
    if(cols != (x_value.shape)[0]):
        x_interval = (max_x - min_x) / (cols)
        x_value = np.arange(min_x + x_interval, max_x, x_interval)
    y_interval = (max_y - min_y) / (rows + 1)
    y_value = np.arange(max_y - y_interval, min_y, -y_interval)
    if(rows != (y_value.shape)[0]):
        y_interval = (max_y - min_y) / (rows)
        y_value = np.arange(max_y - y_interval, min_y, -y_interval)

    x_axis[:, ] = x_value
    y_axis[:, ] = y_value
    y_axis = y_axis.T

    x_fc = x_axis.T[fc.T == 255]
    y_fc = y_axis.T[fc.T == 255]

    return (x_fc, y_fc)

In [63]:
# generate excel file
import xlsxwriter
import numpy as np 

# pic_name = "Test.PNG"
# pic_name = "Q14_e.png"
# res = get_points(pic_name, 2020, 1850, 0.55, -0.7)
pic_name = "Untitled.png"
res = get_points(pic_name, 2020, 1880, 150, 60, 25, 366)
# pic_name = "15_Extreme weather events - ps-x0-83-y0-386.png"
# res = get_points(pic_name, 2020, 1880, 150, 60, 83, 386)
# pic_name = "70_Standardized Precipitation Index (SPI).png"
# res = get_points(pic_name, 2020, 1880, 150, 60)
# pic_name = "20181214_fig4.jpg"
# res = get_points(pic_name, 2018, 1985, 7, 0)
# pic_name = "temperature.png"
# res = get_points(pic_name, 2020, 1, 2, -1)
# pic_name = "obs_global_tc_3.png"
# res = get_points(pic_name, 2020, 1960, 40, 10)

book = xlsxwriter.Workbook(pic_name+".xlsx")
sheet = book.add_worksheet();
sheet.write(0, 0, "x_axis");
sheet.write(0, 1, "y_axis");

xValue = -999999;
row = 1;
points = np.array(res);
[rows, cols] = points.shape
for j in range(cols):
  if np.floor(points[0][j]) > xValue:
    sheet.write(row, 0, np.floor(points[0][j]));
    sheet.write(row, 1, points[1][j]);
    xValue = np.floor(points[0][j]);
    row += 1;

sheet = book.add_worksheet();
sheet.insert_image('A1', pic_name);
book.close();

binary_img
[[  0 255 255 ... 255 255 255]
 [  0 255 255 ... 255 255 255]
 [  0 255 255 ... 255 255 255]
 ...
 [  0 255 255 ... 255 255 255]
 [  0 255 255 ... 255 255 255]
 [  0   0   0 ...   0   0   0]]
(370, 594)
列：

[19125 21675 23970 24480 86445]
[536  23 538 537   0]
行：

[ 43095  43095  84405  84915 151470]
[202 200 328 287 369]
[ 6630  6885  7140  7395  8415  8670  9180 10710 10965 85935]
[ 20  21 537  25 535 536 534  23  22   1]
[  6885   6885   7395  14025  15300  27030  41820  42075 136680 149940]
[367 369 366  15 162 168 250 199 365 368]


ValueError: could not broadcast input array from shape (369,) into shape (594,370)